In [19]:
import pandas as pd
import numpy as np

In [6]:
!ls ../data

case_distribution.csv


In [20]:
dataset = pd.read_csv('../data/case_distribution.csv')
dataset.tail()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
9917,25/03/2020,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
9918,24/03/2020,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0
9919,23/03/2020,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
9920,22/03/2020,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0
9921,21/03/2020,21,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0


## Functions

In [17]:
def muTransform(zMedian):
    return np.log(zMedian)

In [43]:
def sigmaTransform(zMean, mu):
    return np.sqrt(2*(np.log(zMean)-mu))

## Parameters

In [53]:
# setting the baseline CFR
cCFRBaseline = 1.4
cCFREstimateRange = (1.2, 1.7)
# lower end of the range
zmeanLow = 8.7
zmedianLow = 6.7
# middle of the range
zmeanMid = 13
zmedianMid = 9.1
# upper end of the range
zmeanHigh = 20.9
zmedianHigh = 13.7

In [54]:
muLow=muTransform(zmedianLow)
sigmaLow = sigmaTransform(zmeanLow, muLow)
muLow, sigmaLow

(1.9021075263969205, 0.7227938838474179)

In [55]:
muMid = muTransform(zmedianMid)
sigmaMid = sigmaTransform(zmeanMid, muMid)
muMid, sigmaMid

(2.2082744135228043, 0.8446004309005916)

In [56]:
muHigh = muTransform(zmedianHigh)
sigmaHigh = sigmaTransform(zmeanHigh, muHigh)
muHigh, sigmaHigh

(2.617395832834079, 0.9190792415637358)